In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists, TripletDataset
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error
from scipy.sparse import coo_matrix

import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

import torch 
import torch.nn as nn 
import pytorch_lightning as pl
number_of_users, number_of_movies = (10000, 1000)

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
data_pd = pd.read_csv('../data/data_train.csv')
data_matrix, _ = extract_matrix_users_movies_ratings(data_pd)
# axis 0 means: we subtract the means for each movie

data_matrix[data_matrix == 0] = 'nan'
MEAN = np.nanmean(data_matrix, keepdims=True)
MEAN_USER_MATRIX = model.user_bias.weight.detach().numpy()
MEAN_MOVIE_MATRIX = model.movie_bias.weight.detach().numpy()
data_matrix[np.isnan(data_matrix)] = 0


data_matrix_normalized = np.subtract(data_matrix, MEAN, where=data_matrix != 0)
#MEAN_MOVIE_MATRIX = np.nanmean(data_matrix_normalized, axis=0, keepdims=True)
data_matrix_normalized = np.subtract(data_matrix_normalized, MEAN_MOVIE_MATRIX.T, where=data_matrix != 0)
#MEAN_USER_MATRIX = np.nanmean(data_matrix_normalized, axis=1, keepdims=True)
data_matrix_normalized = np.subtract(data_matrix_normalized, MEAN_USER_MATRIX, where=data_matrix != 0)
#STD = np.nanstd(data_matrix, keepdims=True)

STD_MATRIX = np.nanstd(data_matrix, axis=0, keepdims=True)

In [2]:
data_pd = pd.read_csv('../data/data_train.csv')
data_matrix, _ = extract_matrix_users_movies_ratings(data_pd)
data_matrix_normalized = data_matrix

In [3]:
class LearnBias(pl.LightningModule):

    def __init__(self, alpha=0.1, lr=0.001, weight_decay=0):
        super().__init__()
        self.save_hyperparameters()
        
        self.user_bias = nn.Embedding(number_of_users, 1)
        self.movie_bias = nn.Embedding(number_of_movies, 1)
        
        self.lr = lr
        self.weight_decay = weight_decay
        self.alpha = alpha
        

    def forward(self, batch):
        users, movies = batch[:, 0], batch[:, 1]

        user_bias = self.user_bias(users)
        movie_bias = self.movie_bias(movies)
     
        return torch.tensor(MEAN) + self.user_bias(users) + self.movie_bias(movies)
    
    def loss(self, yhat, y):
        return torch.sum((yhat -y)**2) + self.alpha * ((self.user_bias.weight ** 2).sum() + (self.movie_bias.weight ** 2).sum())

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.lr, weight_decay=self.weight_decay)
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        yhat = self(x)
        loss = self.loss(yhat, y)
        self.log('train_rmse', loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x,y = batch
        yhat = self(x)
        loss = self.loss(yhat, y)
        self.log('val_rmse', loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss


In [4]:
BATCH_SIZE=2048

users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(data_pd)
d_train = TripletDataset(users_train, movies_train, ratings_train)
train_dataloader = torch.utils.data.DataLoader(d_train, batch_size=BATCH_SIZE, drop_last=False, shuffle=True)

model = LearnBias()
trainer = pl.Trainer(max_epochs=5)
trainer.fit(model, train_dataloaders=train_dataloader)

/home/david/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/david/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name       | Type      | Params
-----------------------------------------
0 | user_bias  | Embedding | 10.0 K
1 | movie_bias | Embedding | 1.0 K 
-

Training: 0it [00:00, ?it/s]

In [3]:
def normalize_extract_lists(pd, users, movies):  
    # axis 0 means: we subtract the means for each movie
    a = np.zeros(len(users))
    #data_matrix_normalized = np.divide(data_matrix_normalized, STD_MATRIX, where=data_matrix != 0)

    for i, (user, movie) in enumerate(zip(users, movies)):
        a[i] = data_matrix_normalized[user][movie]
    return a

In [4]:
def normalize_lists(pd, users, movies):
    data_matrix, _ = extract_matrix_users_movies_ratings(pd)
    # axis 0 means: we subtract the means for each movie
    a = np.zeros(len(users))
    data_matrix_normalized = np.subtract(data_matrix, MEAN, where=data_matrix != 0)
    #data_matrix_normalized = np.divide(data_matrix_normalized, STD, where=data_matrix != 0)
        
    for i, (user, movie) in enumerate(zip(users, movies)):
        a[i] = data_matrix_normalized[user][movie]
    return a

In [5]:
RANDOM_STATE = 58
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)

users_train, movies_train, _ = extract_users_movies_ratings_lists(train_pd)
ratings_train = normalize_extract_lists(train_pd, users_train, movies_train)

users_val, movies_val, _ = extract_users_movies_ratings_lists(val_pd)
ratings_val = normalize_extract_lists(val_pd, users_val, movies_val)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})

# %%
train_data, data_info = DatasetPure.build_trainset(train)
eval_data = DatasetPure.build_evalset(val)

In [6]:
def get_mean_movie(i):
    mean_vector = np.squeeze(MEAN_MOVIE_MATRIX)
    return mean_vector[i]

def get_mean_user(i):
    mean_vector = np.squeeze(MEAN_USER_MATRIX)
    return mean_vector[i]


def get_std(i):
    std_vector = np.squeeze(STD_MATRIX)
    return std_vector[i]

def get_mean_movie_offset(movies):
    return np.array(list(map(get_mean_movie, movies)))

def get_mean_user_offset(users):
    return np.array(list(map(get_mean_user, users)))


def get_std_offset(movies):
    return np.array(list(map(get_std, movies)))

def transform_back(ratings, users, movies):
    return ratings + MEAN + get_mean_movie_offset(movies) + get_mean_user_offset(users)

In [7]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [8]:
emb_sizes = [2,3,4,6,8,10,12,15]
regs = [0.01, 0.05, 0.1, 0.3, 0.5, 1,2, 3]

david = np.zeros((len(emb_sizes)*len(regs), len(users_val)))

for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                    reg=reg, seed=42)
        giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8,
                eval_data=eval_data, metrics=["rmse", "mae", "r2"])
        david[len(regs)*i+j] = np.array(giorgio.predict(user=users_val, item=movies_val))

            
mean_squared_error(david.mean(axis=0), ratings_val, squared=False)

Training start time: 2022-07-28 17:10:42
Epoch 1 elapsed: 0.046s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 857.42it/s]


	 eval rmse: 2.5230
	 eval mae: 2.1942
	 eval r2: -4.0973
Epoch 2 elapsed: 0.055s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 743.39it/s]


	 eval rmse: 1.0044
	 eval mae: 0.8079
	 eval r2: 0.1922
Epoch 3 elapsed: 0.063s


...

	 eval rmse: 1.0460
	 eval mae: 0.8208
	 eval r2: 0.1238


Epoch 5 elapsed: 0.246s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 628.89it/s]

	 eval rmse: 1.0468
	 eval mae: 0.8198
	 eval r2: 0.1225


0.9801148189680453

In [99]:
transform_back(ratings_val, users_val, movies_val)[0].max()

5.0000002682209015

In [9]:
#------------- submission ----------------#

emb_sizes = [2,3,4,5,6,8,10,12,15]
regs = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1,2, 3]
train_pd = pd.read_csv(f'../data/data_train.csv')

users_train, movies_train, _ = extract_users_movies_ratings_lists(train_pd)
ratings_train = normalize_extract_lists(train_pd, users_train, movies_train)

test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})

david = np.zeros((len(emb_sizes)*len(regs), len(users_test)))

# %%
train_data, data_info = DatasetPure.build_trainset(train)

for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                    reg=reg, seed=42)
        giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
        david[len(regs)*i+j] = np.array(giorgio.predict(user=users_test, item=movies_test))
        
yhat = david.mean(axis=0)
save_predictions(f'./ALS-predictedSubmission.csv', yhat)

Training start time: 2022-07-28 17:12:35
Epoch 1 elapsed: 0.026s
Epoch 2 elapsed: 0.026s
Epoch 3 elapsed: 0.061s
Epoch 4 elapsed: 0.026s
Epoch 5 elapsed: 0.025s
Training start time: 2022-07-28 17:12:36
Epoch 1 elapsed: 0.040s
Epoch 2 elapsed: 0.026s
Epoch 3 elapsed: 0.026s
Epoch 4 elapsed: 0.026s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-28 17:12:37
Epoch 1 elapsed: 0.026s
Epoch 2 elapsed: 0.026s
Epoch 3 elapsed: 0.026s
Epoch 4 elapsed: 0.027s
Epoch 5 elapsed: 0.030s
Training start time: 2022-07-28 17:12:38
Epoch 1 elapsed: 0.031s
Epoch 2 elapsed: 0.025s
Epoch 3 elapsed: 0.025s
Epoch 4 elapsed: 0.049s
Epoch 5 elapsed: 0.025s
Training start time: 2022-07-28 17:12:39
Epoch 1 elapsed: 0.038s
Epoch 2 elapsed: 0.035s
Epoch 3 elapsed: 0.026s
Epoch 4 elapsed: 0.025s
Epoch 5 elapsed: 0.025s
Training start time: 2022-07-28 17:12:39
Epoch 1 elapsed: 0.025s
Epoch 2 elapsed: 0.056s
Epoch 3 elapsed: 0.038s
Epoch 4 elapsed: 0.025s
Epoch 5 elapsed: 0.071s
Training start time: 2022-07-28 17

Epoch 4 elapsed: 0.113s
Epoch 5 elapsed: 0.076s
Training start time: 2022-07-28 17:13:01
Epoch 1 elapsed: 0.069s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.075s
Epoch 4 elapsed: 0.078s
Epoch 5 elapsed: 0.059s
Training start time: 2022-07-28 17:13:02
Epoch 1 elapsed: 0.063s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.062s
Epoch 4 elapsed: 0.074s
Epoch 5 elapsed: 0.059s
Training start time: 2022-07-28 17:13:04
Epoch 1 elapsed: 0.059s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.068s
Epoch 4 elapsed: 0.084s
Epoch 5 elapsed: 0.106s
Training start time: 2022-07-28 17:13:05
Epoch 1 elapsed: 0.083s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.065s
Epoch 4 elapsed: 0.070s
Epoch 5 elapsed: 0.061s
Training start time: 2022-07-28 17:13:06
Epoch 1 elapsed: 0.088s
Epoch 2 elapsed: 0.065s
Epoch 3 elapsed: 0.064s
Epoch 4 elapsed: 0.104s
Epoch 5 elapsed: 0.085s
Training start time: 2022-07-28 17:13:07
Epoch 1 elapsed: 0.081s
Epoch 2 elapsed: 0.060s
Epoch 3 elapsed: 0.061s
Epoch 4 elapsed: 0.062s
Epoch 5 el

Training start time: 2022-07-28 17:13:36
Epoch 1 elapsed: 0.136s
Epoch 2 elapsed: 0.159s
Epoch 3 elapsed: 0.158s
Epoch 4 elapsed: 0.167s
Epoch 5 elapsed: 0.228s
Training start time: 2022-07-28 17:13:38
Epoch 1 elapsed: 0.139s
Epoch 2 elapsed: 0.138s
Epoch 3 elapsed: 0.147s
Epoch 4 elapsed: 0.137s
Epoch 5 elapsed: 0.136s
Training start time: 2022-07-28 17:13:39
Epoch 1 elapsed: 0.140s
Epoch 2 elapsed: 0.161s
Epoch 3 elapsed: 0.156s
Epoch 4 elapsed: 0.182s
Epoch 5 elapsed: 0.169s
Training start time: 2022-07-28 17:13:41
Epoch 1 elapsed: 0.162s
Epoch 2 elapsed: 0.176s
Epoch 3 elapsed: 0.171s
Epoch 4 elapsed: 0.150s
Epoch 5 elapsed: 0.139s
Training start time: 2022-07-28 17:13:42
Epoch 1 elapsed: 0.136s
Epoch 2 elapsed: 0.139s
Epoch 3 elapsed: 0.173s
Epoch 4 elapsed: 0.144s
Epoch 5 elapsed: 0.154s
Training start time: 2022-07-28 17:13:44
Epoch 1 elapsed: 0.231s
Epoch 2 elapsed: 0.230s
Epoch 3 elapsed: 0.201s
Epoch 4 elapsed: 0.208s
Epoch 5 elapsed: 0.220s
Training start time: 2022-07-28 17

In [10]:
yhat

array([3.22483046, 3.22381977, 3.81900809, ..., 3.01826393, 3.06777642,
       3.76851724])

In [11]:
yhat.std()

0.579380095099008